## Problem Description
____
Determine which cities to purchase a new larger, more fuel efficient truck in to gain full coverage of all cities. The new trucks will also be available to cities within 400 miles of the city with the new truck.

Table of distances between cities

|Miles        |Boston |New York |Philadelphia |Baltimore |Washington |Richmond |Raleigh |Florence |Savannah |Jacksonville |Tampa  |Miami |
|-------------|-------|---------|-------------|----------|-----------|---------|--------|---------|---------|-------------|-------|------|
|Boston       |	 -    |	 211 	|  320 	      | 424 	 | 459 	     | 565 	   | 713 	| 884 	  | 1,056 	| 1,196 	  | 1,399 | 1,669| 
|New York     |	 211  |	 -   	|  109 	      | 213 	 | 248 	     | 354 	   | 502 	| 673 	  | 845 	| 985 	      | 1,188 |	1,458| 
|Philadelphia |	 320  |	 109 	|  -   	      | 104 	 | 139 	     | 245 	   | 393 	| 564 	  | 736 	| 876 	      | 1,079 |	1,349| 
|Baltimore    |	 424  |	 213 	|  104 	      | -   	 | 35 	     | 141 	   | 289 	| 460 	  | 632 	| 772 	      | 975   |	1,245| 
|Washington	  |  459  |  248 	|  139 	      | 35 	     | -   	     | 106 	   | 254 	| 425 	  | 597 	| 737 	      | 940   |	1,210| 
|Richmond	  |  565  |  354    |  245 	      | 141 	 | 106 	     | -   	   | 148 	| 319 	  | 491 	| 631 	      | 834   |	1,104| 
|Raleigh	  |  713  |  502    |  393 	      | 289 	 | 254 	     | 148 	   | -   	| 171 	  | 343 	| 483 	      | 686   |	956  |
|Florence	  |  884  |  673    |  564 	      | 460 	 | 425 	     | 319 	   | 171 	| -   	  | 172 	| 312 	      | 515   |	785  | 
|Savannah	  | 1,056 |  845    |  736 	      | 632 	 | 597 	     | 491 	   | 343 	| 172 	  | -   	| 140 	      | 343   |	613  | 
|Jacksonville |	1,196 |  985    |  876 	      | 772 	 | 737 	     | 631 	   | 483 	| 312 	  | 140 	| -   	      | 203   |	473  | 
|Tampa	      | 1,399 | 1,188   | 1,079 	  | 975 	 | 940 	     | 834 	   | 686 	| 515 	  | 343 	| 203 	      | -     |	270  | 
|Miami        | 1,669 | 1,458   | 1,349 	  | 1,245 	 | 1,210 	 | 1,104   | 956 	| 785 	  | 613 	| 473 	      | 270   |	-    |



In [1]:
import numpy as np 
import pandas as pd
import pulp

In [11]:
#fixed variables
cities = ['Boston','New York','Philadelphia','Baltimore','Washington','Richmond','Raleigh','Florence','Savannah',
            'Jacksonville','Tampa','Miami']
max_distance = 400
distances = np.array([[0,211,320,424,459,565,713,884,1056,1196,1399,1669],
            [211,0,109,213,248,354,502,673,845,985,1188,1458],
            [320,109,0,104,139,245,393,564,736,876,1079,1349],
            [424,213,104,0,35,141,289,460,632,772,975,1245],
            [459,248,139,35,0,106,254,425,597,737,940,1210],
            [565,354,245,141,106,0,148,319,491,631,834,1104],
            [713,502,393,289,254,148,0,171,343,483,686,956],
            [884,673,564,460,425,319,171,0,172,312,515,785],
            [1056,845,736,632,597,491,343,172,0,140,343,613],
            [1196,985,876,772,737,631,483,312,140,0,203,473],
            [1399,1188,1079,975,940,834,686,515,343,203,0,270],
            [1669,1458,1349,1245,1210,1104,956,785,613,473,270,0]])

n_cities=len(cities)

nearby_cities = np.array([1 if distances[i,j]<=max_distance else 0 for i in range(n_cities) for j in range(n_cities)]).reshape(n_cities,n_cities)



In [26]:
#create the model
model = pulp.LpProblem(name='truck_purchase_optimization', sense=pulp.LpMinimize)


city_variables = pulp.LpVariable.matrix('C', cities,
                                             lowBound=0, cat=pulp.LpBinary)

#Sum of each city row must be at least 1
for index, city in enumerate(cities):
    model.addConstraint(pulp.LpConstraint(
        e=pulp.lpSum(nearby_cities[index,:]*city_variables) ,
        sense=pulp.LpConstraintGE,
        name='min_trucks_'+city,
        rhs=1))

objective = pulp.lpSum(city_variables)

model.setObjective(objective)

model.solve()

if model.status == 1:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
    print(f'objective: ${model.objective.value():,.0f}')
    for i,var in enumerate(city_variables):
        print(f'{var}: {var.value()}')
else:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')

status: 1, Optimal
objective: $3
C_Boston: 1.0
C_New_York: 0.0
C_Philadelphia: 0.0
C_Baltimore: 0.0
C_Washington: 0.0
C_Richmond: 1.0
C_Raleigh: 0.0
C_Florence: 0.0
C_Savannah: 0.0
C_Jacksonville: 0.0
C_Tampa: 1.0
C_Miami: 0.0


In [28]:
#run a second model to maximize the number of cities that the new trucks will cover, this is essential a second objective function
model = pulp.LpProblem(name='round2_optimization', sense=pulp.LpMaximize)

city_variables = pulp.LpVariable.matrix('C', cities,
                                             lowBound=0, cat=pulp.LpBinary)

#Sum of each city row must be at least 1
for index, city in enumerate(cities):
    model.addConstraint(pulp.LpConstraint(
        e=pulp.lpSum(nearby_cities[index,:]*city_variables) ,
        sense=pulp.LpConstraintGE,
        name='min_trucks_'+city,
        rhs=1))

#add the constraint from the first model run, sum of city_variables = 3
model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(city_variables) ,
    sense=pulp.LpConstraintEQ,
    name='obj_1',
    rhs=3))

objective = pulp.lpSum(city_variables*nearby_cities)

model.setObjective(objective)

model.solve()

if model.status == 1:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
    print(f'objective: ${model.objective.value():,.0f}')
    for i,var in enumerate(city_variables):
        print(f'{var}: {var.value()}')
else:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')

status: 1, Optimal
objective: $18
C_Boston: 0.0
C_New_York: 0.0
C_Philadelphia: 1.0
C_Baltimore: 0.0
C_Washington: 0.0
C_Richmond: 1.0
C_Raleigh: 0.0
C_Florence: 0.0
C_Savannah: 0.0
C_Jacksonville: 0.0
C_Tampa: 1.0
C_Miami: 0.0


In [29]:
model

round2_optimization:
MAXIMIZE
6*C_Baltimore + 3*C_Boston + 5*C_Florence + 4*C_Jacksonville + 2*C_Miami + 6*C_New_York + 7*C_Philadelphia + 7*C_Raleigh + 7*C_Richmond + 5*C_Savannah + 4*C_Tampa + 6*C_Washington + 0
SUBJECT TO
min_trucks_Boston: C_Boston + C_New_York + C_Philadelphia >= 1

min_trucks_New_York: C_Baltimore + C_Boston + C_New_York + C_Philadelphia
 + C_Richmond + C_Washington >= 1

min_trucks_Philadelphia: C_Baltimore + C_Boston + C_New_York + C_Philadelphia
 + C_Raleigh + C_Richmond + C_Washington >= 1

min_trucks_Baltimore: C_Baltimore + C_New_York + C_Philadelphia + C_Raleigh
 + C_Richmond + C_Washington >= 1

min_trucks_Washington: C_Baltimore + C_New_York + C_Philadelphia + C_Raleigh
 + C_Richmond + C_Washington >= 1

min_trucks_Richmond: C_Baltimore + C_Florence + C_New_York + C_Philadelphia
 + C_Raleigh + C_Richmond + C_Washington >= 1

min_trucks_Raleigh: C_Baltimore + C_Florence + C_Philadelphia + C_Raleigh
 + C_Richmond + C_Savannah + C_Washington >= 1

min_truck